In [18]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [19]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [21]:
genai.configure(api_key=GOOGLE_API_KEY)

In [22]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [24]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [25]:
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

**The Ballad of LangChain**

In realms of code, where logic reigns,
A tale unfolds, a digital refrain.
LangChain, a bard, with words so grand,
A generative mind, at its command.

From depths of data, it weaves its rhyme,
A symphony of text, in perfect time.
With every prompt, a story takes flight,
A tapestry of words, a dazzling sight.

Through prompts of love, or fear, or strife,
LangChain weaves tales that dance with life.
It paints emotions with vibrant hues,
And crafts adventures that captivate and amuse.

But with great power comes a weighty task,
To wield its words with care, a sacred mask.
For language, wielded well, can heal or harm,
And LangChain's words hold an uncanny charm.

So let us sing its praise, this digital bard,
Whose words ignite our hearts, and leave their mark.
May LangChain's legacy forever gleam,
A testament to AI and its boundless dream.

But let us also heed the bard's decree,
To use its power wisely, with integrity.
For in the realm of words, both sweet and 

In [26]:
chroma = Chroma(persist_directory="../raw_data/chroma_db", embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

In [27]:
print(chroma)

In [28]:
len(chroma.get()['documents'])

22923

In [29]:
retriever = chroma.as_retriever()

In [30]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f6db8709930>)

In [31]:
query = "Tell me horror movies with zombies"
docs = chroma.similarity_search(query)
docs

[Document(page_content='id: 1043771\nTitle: The Curse of the Screaming Dead\nYear: 1982.0\nDescription: A group of six friends are out in the rural south on a deer hunt when they start hearing strange noises coming from the woods. An investigation of the sounds leads them to an old Confederate graveyard, which suddenly begins sprouting Rebel zombies. Like the living dead in every other movie, these zombies are hungry for human flesh and soon set to chowing down on the hapless hunters.\nMinutes: 91.0\nRating: 2.52\nGenre: Horror\nActors: Richard Ruxton, Jim Ball, Mimi Ishikawa, Mark Redfield, Judy Dixon, Bumb Roberts, Rebecca Bach, Christopher Gummer, Steve Sandkuhler\nTheme: Bloody vampire horror, Gothic and eerie haunting horror, Gory, gruesome, and slasher horror, Horror, the undead and monster classics, Survival horror and zombie carnage, Monsters, aliens, sci-fi and the apocalypse, Extreme gory horror and cannibalism', metadata={'row': 20127, 'source': 'movies_data.csv'}),
 Documen

In [32]:
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"Who are Star Wars actors?"})
#Print the answer to the question
print(response["answer"])

- James Earl Jones
- David Prowse
- Peter Mayhew
- Kenny Baker
- Anthony Daniels
- Alec Guinness
- Peter Cushing
- Carrie Fisher
- Harrison Ford
- Mark Hamill
